In [ ]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir

    path.append(dir(path[0]))
    __package__ = "dash_app"

In [ ]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map, map_columns
from settings import *


In [ ]:
data_path = DATA_PATH+'Atmospheric_Domain/2.3WaterVapour/Figure2.6/'
data_csv = data_path + 'Figure2.6_data.csv'

In [ ]:
"""
Tidy data for charts
"""
#!!!! takes long time
xls = pd.ExcelFile(
    data_path+'MonthlyAndAnnualAverageWaterVapourPressure_ValentiaObservatory2019.xlsx')
original_df = pd.read_excel(xls, 'Valentia')
filtered_df = original_df.drop(original_df[original_df.year < 1940].index)
# filtered_df = filtered_df.drop(filtered_df[filtered_df.year > 2018].index)
filtered_df['datetime']=filtered_df['year'].astype(str)+'-'+filtered_df['month'].astype(str)+'-'+filtered_df['day'].astype(str)+'T'+filtered_df['hour'].astype(str)+':00:00'
filtered_df['datetime'] = pd.to_datetime(filtered_df['datetime'])
# Create series with datetime as index
vp_series =pd.Series(filtered_df['vp'].values,index=filtered_df['datetime'])
# Calc mean annual 
vp_mean_annual = vp_series.resample('Y').mean()
# Calc mean monthly 
vp_mean_monthly = vp_series.resample('M').mean()

# Create DF
annual_df = pd.DataFrame()
annual_df['mean__annual__water_vapour_pressure'] = vp_mean_annual
monthly_df = pd.DataFrame()
monthly_df['mean__monthly__water_vapour_pressure'] = vp_mean_monthly
df = pd.concat([monthly_df,annual_df])
df['datetime'] = df.index
df.reset_index(drop=True, inplace=True)
# rearrange collumns
df = df[['datetime','mean__monthly__water_vapour_pressure','mean__annual__water_vapour_pressure']]
df['location']='Valentia'
# df.to_csv(data_csv) #commented out as the file should be kept as is


In [ ]:
data_path = DATA_PATH+'Atmospheric_Domain/2.3WaterVapour/Figure2.6/'
data_csv = data_path + 'Figure2.6_data.csv'
df = pd.read_csv(data_csv, index_col=0)
df['datetime'] = pd.to_datetime(df['datetime'])
df

In [ ]:
mean_annual_trace = go.Scatter(x=df['datetime'],
                            y=df['mean__annual__water_vapour_pressure'],
                         name='Mean Annual',
                         mode='markers+lines',
                         marker=dict(color=TIMESERIES_COLOR_2,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_2,
                                      width=1),
                         hovertemplate='%{x|%Y}<br>' +
                         '<b>Mean Annual</b><br>' +
                         'Vapour Pressure: %{y:.2f} hPa<br>' +
                         '<extra></extra>'
                         )
figure_2_6_b = go.Figure(data=[mean_annual_trace], layout=TIMESERIES_LAYOUT)
figure_2_6_b.update_layout(
    yaxis=dict(title='Vapour Presure (hPa)'),
    xaxis=dict(title="Year"))
figure_2_6_b

In [ ]:
months=[1,2,3,4,5,6,7,8,9,10,11,12]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
month_mean = []
df_months = df.groupby(df['datetime'].dt.month).mean()
df_months

In [ ]:
month_trace = go.Scatter(x=months,
                            y=df_months['mean__monthly__water_vapour_pressure'],
                         name='Monthly Average (1940-2019)',
                         mode='markers+lines',
                         marker=dict(color=TIMESERIES_COLOR_1,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_1,
                                      width=1),
                         hovertemplate='%{x}<br>' +
                         '<b>Monthly Average (1940-2019)</b><br>' +
                         'Vapour Pressure: %{y:.2f} hPa<br>' +
                         '<extra></extra>'
                         )
figure_2_6_a = go.Figure(data=[month_trace], layout=TIMESERIES_LAYOUT)
figure_2_6_a.update_layout(
    yaxis=dict(title='Vapour Presure (hPa)'),
    xaxis=dict(title="Month"))
figure_2_6_a